## 딥러닝 실전!! 영화 리뷰가 긍정적인지 부정적인지 예측하는 예제를 통해 실전을 다져보자!


### 우선 필요한 라이브러리를 설치, 호출한다.

In [181]:
!pip install tensorflow
!pip install keras

ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\TJ\\anaconda3\\envs\\HJK\\Lib\\site-packages\\numpy\\.libs\\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.




  Using cached numpy-1.19.5-cp38-cp38-win_amd64.whl (13.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3
  Rolling back uninstall of numpy
  Moving to c:\users\tj\anaconda3\envs\hjk\lib\site-packages\numpy-1.20.3.dist-info\
   from C:\Users\TJ\anaconda3\envs\HJK\Lib\site-packages\~umpy-1.20.3.dist-info
  Moving to c:\users\tj\anaconda3\envs\hjk\lib\site-packages\numpy\__config__.py
   from C:\Users\TJ\AppData\Local\Temp\pip-uninstall-sdt6mx2e\__config__.py
  Moving to c:\users\tj\anaconda3\envs\hjk\lib\site-packages\numpy\__init__.cython-30.pxd
   from C:\Users\TJ\AppData\Local\Temp\pip-uninstall-sdt6mx2e\__init__.cython-30.pxd
  Moving to c:\users\tj\anaconda3\envs\hjk\lib\site-packages\numpy\__init__.pxd
   from C:\Users\TJ\AppData\Local\Temp\pip-uninstall-sdt6mx2e\__init__.pxd
  Moving to c:\users\tj\anaconda3\envs\hjk\lib\site-packages\numpy\__init__.py
   from C:\Users\TJ\A

In [182]:
import numpy
import tensorflow as tf 
from numpy import array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding

## 필요 라이브러리를 모두 불러왔다. 이제 텍스트 리뷰 자료를 긍정과 부정 쪽으로 작성하고 지정한다.

In [183]:
# 일단 긍정, 부정 순으로 텍스트 리뷰 자료 지정.

docs = ['너무 재밌네요', '최고에요', '참 잘 만든 영화에요', '추천하고 싶은 영화입니다.', '한번 더 보고싶네요', 
        '글쎄요', '별로에요', '생각보다 지루해요', '연기가 어색해요', '재미없어요']

# 긍정 리뷰는 1, 부정 리뷰는 0으로 클래스 지정
        
classes = array([1,1,1,1,1,0,0,0,0,0])

In [184]:
# 토큰화

token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_index)

{'너무': 1, '재밌네요': 2, '최고에요': 3, '참': 4, '잘': 5, '만든': 6, '영화에요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로에요': 15, '생각보다': 16, '지루해요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}


In [185]:
# 토큰에 지정된 인덱스로 새로운 배열을 생성한다.

x = token.texts_to_sequences(docs)
print(x)

[[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13], [14], [15], [16, 17], [18, 19], [20]]


In [186]:
# 패딩, 서로 다른 길이의 데이터를 4로 맞춤.

padded_x = pad_sequences(x, 4)
"\n패딩 결과\n", print(padded_x)

[[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [ 0 11 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0  0 20]]


('\n패딩 결과\n', None)

## 여기서 패딩 결과를 출력해보면, 각 단어가 1부터 20가지의 숫자로 토큰화되었다는 걸 파악할 수 있다.
## 그런데 리뷰 데이터의 토큰 수가 텍스트 갯수에 따라 각각 다르다느 것을 유의해야 한다.
## 예를 들어 '최고에요'는 하나의 토큰([3])이지만 ' 참 잘 만든 영화에요' 는 4개의 토큰데이터(4[,5,6,7])가 나온다.

### 따라서 이 딥러닝 모델에 입력을 제대로 하려면 학습 데이터의 길이가 동일해야한다는 전제조건이 붙는다.
### 이처럼 길이를 맞춰주는 작업을 패딩(padding)이라고 하고, 이건 자연어 처리 뿐만 아니라 GAN에서도 중요한 역할을 하니 기억할 것.

In [188]:
word_size = len(token.word_index) + 1

In [189]:
Embedding(word_size, 8, input_length=4)

In [192]:
# 단어 임베딩을 포함하여 딥러닝 모델을 만들고 결과를 출력.

model = Sequential()
model.add(Embedding(word_size, 8, input_length=4))
model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(padded_x, classes, epochs=20)
print("\n Accuracy: %.4f" % (model.evaluate(padded_x, labels)[1]))

Epoch 1/20
1/1 [==============================] - 0s 323ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 2/20
1/1 [==============================] - 0s 3ms/step - loss: 0.6908 - accuracy: 0.5000
Epoch 3/20
1/1 [==============================] - 0s 3ms/step - loss: 0.6884 - accuracy: 0.6000
Epoch 4/20
1/1 [==============================] - 0s 2ms/step - loss: 0.6860 - accuracy: 0.7000
Epoch 5/20
1/1 [==============================] - 0s 1ms/step - loss: 0.6836 - accuracy: 0.7000
Epoch 6/20
1/1 [==============================] - 0s 3ms/step - loss: 0.6813 - accuracy: 0.7000
Epoch 7/20
1/1 [==============================] - 0s 2ms/step - loss: 0.6789 - accuracy: 0.8000
Epoch 8/20
1/1 [==============================] - 0s 3ms/step - loss: 0.6766 - accuracy: 0.8000
Epoch 9/20
1/1 [==============================] - 0s 3ms/step - loss: 0.6742 - accuracy: 0.9000
Epoch 10/20
1/1 [==============================] - 0s 3ms/step - loss: 0.6718 - accuracy: 1.0000
Epoch 11/20
1/1 [====================

ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, <class 'map'>

###  학습 결과가 10개의 리뷰 샘플 중 몇 개의 결과를 맞추었는지 보여준다.
###  계산해보면 65% 정도의 정확도를 지닌 accuracy를 얻었다.
